[Oregon Curriculum Network](http://www.4dsolutions.net/ocn) <br />
[Discovering Math with Python](Introduction.ipynb)


# Chapter 7: POLYHEDRONS

POV-Ray understands XYZ coordinates, so at the end of the day we'll need those for rendering our polyhedrons as ray tracings.  On the other hand, building our database of Vertexes might depend on a more comfortable set of 4-tuples.  We will store our vertexes in Quadray Coordinates (see Chapter 6) and convert to XYZ on the fly.

Back to Chapter 6: [Vectors in Space](Vectors%20%in%20Space.ipynb) <br />
Continue to Chapter 8: [ Transformations](Transformations.ipynb)<br />
[Introduction / Table of Contents](Introduction.ipynb)